In [ ]:
import ee
!pip install geemap
import geemap

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=tGo8HM2K9uaR4DIii63jGMV-pL3nWhotMFZP_Uujw2w&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhMcMCbMHyPXTaRGTQqQgoX6EdatEh9Hd-56PNNjiG-du-5ne2eaFs

Successfully saved authorization token.


## NDVI

In [ ]:
aoi = ee.Geometry.Rectangle(-119.19, 13.1,  -84.64, 33.72)
col = ee.ImageCollection('MODIS/006/MOD13A2').filterDate('2002-01-01','2021-06-02').select('NDVI')
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Mexico'))

def agregar_dia_anio(img):
  doy = ee.Date(img.get('system:time_start')).getRelative('day', 'year')
  return img.set('doy', doy)

col = col.map(agregar_dia_anio)

distinctDOY = col.filterDate('2020-01-01', '2021-01-01')
filter = ee.Filter.equals(**{'leftField':'doy', 'rightField':'doy'})
join = ee.Join.saveAll('doy_matches')
joinCol = ee.ImageCollection(join.apply(distinctDOY, col, filter))

def func_zdr(img):
  doyCol = ee.ImageCollection.fromImages(
    img.get('doy_matches')
  )
  return doyCol.reduce(ee.Reducer.max()).copyProperties(img, img.propertyNames())

comp = joinCol.map(lambda img: ee.ImageCollection.fromImages(
    img.get('doy_matches')).reduce(ee.Reducer.mean()))

visParams = {
  'min': 0.0,
  'max': 9000.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}

# RGB visualization 

def func_xbj(img):
  return img.visualize(visParams).clip(mask)

rgbVis = comp.map(lambda img: img.visualize(**visParams).clip(mask).copyProperties(img, img.propertyNames()))

# Arguments for animation 
videoArgs = {
  'region': aoi,
  'dimensions': 900,
  'crs': 'EPSG:6372',
  'framesPerSecond': 10
}

from IPython.display import Image
url = (rgbVis.getVideoThumbURL(videoArgs))
display(Image(url=url))

In [ ]:
print(url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/0490df9f4e3ed93b0a3b71b5aba933d5-0d36e629ed420336651581f0a27ff07b:getPixels


## PRECIPITATION



In [ ]:
aoi = ee.Geometry.Rectangle(-119.19, 13.1,  -84.64, 33.72)

col = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD').filterDate('2002-01-01','2021-06-02').select('precipitation')
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Mexico'))

def agregar_dia_anio(img):
  doy = ee.Date(img.get('system:time_start')).getRelative('day', 'year')
  return img.set('doy', doy)

col = col.map(agregar_dia_anio)

distinctDOY = col.filterDate('2020-01-01', '2021-01-01')
filter = ee.Filter.equals(**{'leftField':'doy', 'rightField':'doy'})
join = ee.Join.saveAll('doy_matches')
joinCol = ee.ImageCollection(join.apply(distinctDOY, col, filter))

def func_zdr(img):
  doyCol = ee.ImageCollection.fromImages(
    img.get('doy_matches')
  )
  return doyCol.reduce(ee.Reducer.max()).copyProperties(img, img.propertyNames())

comp = joinCol.map(lambda img: ee.ImageCollection.fromImages(
    img.get('doy_matches')).reduce(ee.Reducer.mean()))

visParams = {
  'min': 0.0,
  'max': 150.0,
  'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000'],
}

# Visualization
def func_xbj(img):
  return img.visualize(visParams).clip(mask)

rgbVis = comp.map(lambda img: img.visualize(**visParams).clip(mask).copyProperties(img, img.propertyNames()))

# Arguments for animation 
videoArgs = {
  'region': aoi,
  'dimensions': 750,
  'crs': 'EPSG:6372',
  'framesPerSecond': 10
}

from IPython.display import Image
url = (rgbVis.getVideoThumbURL(videoArgs))
display(Image(url=url))

In [ ]:
print(url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/0f1084b5debde295fff3c25ae05102f5-228a4cf93d3cbf2d7341004498a6f331:getPixels


## TEMPERATURE


In [ ]:
aoi = ee.Geometry.Rectangle(-119.19, 13.1,  -84.64, 33.72)

col = ee.ImageCollection('ECMWF/ERA5/MONTHLY').filterDate('2002-01-01','2021-06-02').select('mean_2m_air_temperature')
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Mexico'))

def agregar_dia_anio(img):
  doy = ee.Date(img.get('system:time_start')).getRelative('day', 'year')
  return img.set('doy', doy)

col = col.map(agregar_dia_anio)

distinctDOY = col.filterDate('2020-01-01', '2021-01-01')
filter = ee.Filter.equals(**{'leftField':'doy', 'rightField':'doy'})
join = ee.Join.saveAll('doy_matches')
joinCol = ee.ImageCollection(join.apply(distinctDOY, col, filter))

def func_zdr(img):
  doyCol = ee.ImageCollection.fromImages(
    img.get('doy_matches')
  )
  return doyCol.reduce(ee.Reducer.max()).copyProperties(img, img.propertyNames())

comp = joinCol.map(lambda img: ee.ImageCollection.fromImages(
    img.get('doy_matches')).reduce(ee.Reducer.mean()))

visParams = {
  'min': 280.0,
  'max': 310.0,
  'palette': [
    "#000080","#0000D9","#4000FF","#8000FF","#0080FF","#00FFFF",
    "#00FF80","#80FF00","#DAFF00","#FFFF00","#FFF500","#FFDA00",
    "#FFB000","#FFA400","#FF4F00","#FF2500","#FF0A00","#FF00FF",
  ],
}

# RGB visualization

def func_xbj(img):
  return img.visualize(visParams).clip(mask)

rgbVis = comp.map(lambda img: img.visualize(**visParams).clip(mask).copyProperties(img, img.propertyNames()))

# Arguments for animation 
videoArgs = {
  'region': aoi,
  'dimensions': 750,
  'crs': 'EPSG:6372',
  'framesPerSecond': 1
}

from IPython.display import Image
url = (rgbVis.getVideoThumbURL(videoArgs))
display(Image(url=url))

In [ ]:
print(url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/a20a4fb4ba3e1d91f3f5c2e0ede1e4bf-13575a985414e454a54aa53191cc6a9a:getPixels
